In [1]:
# Importing the Libraries

import json
import praw, prawcore

#### Loading the 'config' json holding the credentials for running the 'reddit' client.

In [2]:
#setting up the 'praw' client
config = json.load(open('./config.json', 'r'))

reddit = praw.Reddit(
    client_id=config['clientId'],
    client_secret=config['clientSecret'],
    user_agent=config['userAgent'],
)



#### Listing the 20 'popular' subreddits and listing the top 20 'active' users in the community. 

In [3]:
# List of popular subreddits
active_users = set()
subreddits = reddit.subreddits.popular(limit=20) #Popular '20' subreddits
for sub in subreddits:
    for submission in sub.new(limit=20): #New '20' Submissions per subreddit.
        if submission.author is not None:
            active_users.add(submission.author.name)

print(len(active_users))

385


#### Temporarily save the user activity (user + subreddits) to a JSON file.

In [4]:
# User's activity 
from collections import defaultdict

active_users = list(active_users)

# fetching the list of a subreddits (submission or comment) for a user
def map_user_activity(username, user_activity):
    user = reddit.redditor(username)

    subreddit_names = {item.subreddit.display_name for item in user.submissions.new(limit=100)}
    subreddit_names |= {item.subreddit.display_name for item in user.comments.new(limit=100)}
    user_activity[username] = list(subreddit_names)
    

user_activity = defaultdict(list)
for user in active_users:
    map_user_activity(user, user_activity)   

# intermediate step -- save to json (Since more number of nodes are stored)
with open('user_activity.json', 'w') as fp:
    json.dump(user_activity, fp)
